In [1]:
import json
import gpxpy
import pandas as pnd
import numpy as np
import rdflib

import folium
import geojson

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from preprocess import myplot

import requests

ImportError: No module named 'gpxpy'

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib notebook
matplotlib.rcParams['axes.labelsize'] = 'large'
matplotlib.rcParams['ytick.labelsize'] = 'medium'
matplotlib.rcParams['xtick.labelsize'] = 'medium'
matplotlib.rcParams['legend.fancybox'] = True
matplotlib.rcParams['figure.figsize'] = (7, 5)
sns.set_style('whitegrid')

requests.packages.urllib3.disable_warnings()

In [3]:
tracks = {'1': r'C:\DATA\MapMyRun\Grenoble\40890583\16_16_Run _ Jog\950590555.gpx',
            '2': r'C:\DATA\MapMyRun\Grenoble\37695677\16_16_Run _ Jog\1007756961.gpx',
            '3': r'C:\DATA\MapMyRun\Grenoble\49996348\16_16_Run _ Jog\1079694145.gpx',
            '4': r'C:\DATA\MapMyRun\Grenoble\27031614\16_283_Easy _ General\1223879101.gpx',
            '5': r'C:\DATA\MapMyRun\Grenoble\41837719\16_16_Run _ Jog\1320895325.gpx',
            '6': r'C:\DATA\MapMyRun\Grenoble\41837719\16_16_Run _ Jog\1234176355.gpx',
            '7': r'C:\DATA\MapMyRun\Grenoble\27031614\16_16_Run _ Jog\728010151.gpx'}
            
gpx = {}
dataframes = {}

for key, path in tracks.items():
    print(key, path)
    g = gpxpy.parse(open(path, 'r'))
    gpx.update({key: g})

7 C:\DATA\MapMyRun\Grenoble\27031614\16_16_Run _ Jog\728010151.gpx
4 C:\DATA\MapMyRun\Grenoble\27031614\16_283_Easy _ General\1223879101.gpx
2 C:\DATA\MapMyRun\Grenoble\37695677\16_16_Run _ Jog\1007756961.gpx
6 C:\DATA\MapMyRun\Grenoble\41837719\16_16_Run _ Jog\1234176355.gpx
1 C:\DATA\MapMyRun\Grenoble\40890583\16_16_Run _ Jog\950590555.gpx
3 C:\DATA\MapMyRun\Grenoble\49996348\16_16_Run _ Jog\1079694145.gpx
5 C:\DATA\MapMyRun\Grenoble\41837719\16_16_Run _ Jog\1320895325.gpx


In [4]:
dict_id = '3'

In [5]:
#myplot.plot_map(gpx[dict_id])

In [6]:
points_data = gpx[dict_id].get_points_data()
first_point = points_data[0].point
coords = str(first_point.latitude) + ', ' + str(first_point.longitude)
trajectory_date = first_point.time
starting_day = "{}-{}-{}".format(trajectory_date.year, trajectory_date.month, trajectory_date.day)
starting_hour = trajectory_date.hour

# Weather Annotation

_PremiumApiKey = '5261d6e39eae4d8f8eb152732160804'
_PremiumApiBaseURL = 'http://api.worldweatheronline.com/premium/v1/'
#url = _PremiumApiBaseURL + 'weather.ashx' 
url = _PremiumApiBaseURL + 'past-weather.ashx'

q = coords
date = starting_day

#Switch between weather forecast time interval from 1 hourly, 3 hourly, 6 hourly, 
#12 hourly (day/night) or 24 hourly (day average). E.g:- tp=24 or tp=12 or tp=6 or tp=3 or tp=1
tp = '1'

parameters = {'q': q, 'format': 'JSON', 'date': date, 'tp': tp, 'key': _PremiumApiKey}
r = requests.get(url, params=parameters)
print r.url

In [7]:
#response = json.loads(r.content)
response = json.load(open(r"C:\DATA\weather 49996348 16_16_Run _ Jog 1079694145.json", 'r'))
#response

In [8]:
starting_weather = response['data']['weather'][0]['hourly'][starting_hour]
print('time:', starting_weather['time'])
print('tempC:', starting_weather['tempC'])
print('FeelsLikeC:', starting_weather['FeelsLikeC'])
print('weatherDesc:', starting_weather['weatherDesc'][0]['value'])
print('winddir16Point:', starting_weather['winddir16Point'])
print('windspeedKmph:', starting_weather['windspeedKmph'])

time: 1700
tempC: 30
FeelsLikeC: 31
weatherDesc: Patchy rain nearby
winddir16Point: S
windspeedKmph: 6


In [9]:
from annotate import step, annotate
from datetime import datetime

weather = step.ContextualElement("Weather")

foi_feelslike = step.FeatureOfInterest("Feels Like")
foi_temperature = step.FeatureOfInterest("Temperature")
foi_wind_dir = step.FeatureOfInterest("Wind Direction")
foi_wind_speed = step.FeatureOfInterest("Wind Speed")

date = datetime(trajectory_date.year, trajectory_date.month, trajectory_date.day, trajectory_date.hour)
instant = step.TimeInstant(date)

#FeelsLikeC
feelslike_description = step.QuantitativeValue(starting_weather['FeelsLikeC'], "C")
foi_feelslike.add_episode(step.Episode(instant, feelslike_description))

#tempC
tempC_description = step.QuantitativeValue(starting_weather['tempC'], "C")
foi_temperature.add_episode(step.Episode(instant, tempC_description))

#winddir16Point
wind_dir_description = step.QualitativeDescription(starting_weather['winddir16Point'])
foi_wind_dir.add_episode(step.Episode(instant, wind_dir_description))

#windspeedKmph
wind_speed_description = step.QuantitativeValue(starting_weather['windspeedKmph'], "km/h")
foi_wind_speed.add_episode(step.Episode(instant, wind_speed_description))


weather.features_of_interest.append(foi_feelslike)
weather.features_of_interest.append(foi_temperature)
weather.features_of_interest.append(foi_wind_dir)
weather.features_of_interest.append(foi_wind_speed)

In [10]:
print(weather)

ContextualElement { label: "Weather", FeaturesOfInterest: [{ FeatureOfInterest { label: "Feels Like", Episodes: [{ Episode { semantic_description:{ QuantitativeValue { value: "31", unit: "C"} }, extent: { TimeInstant { in_datetime: "2015-07-19 17:00:00"} }, relates_to: None } }] } }, { FeatureOfInterest { label: "Temperature", Episodes: [{ Episode { semantic_description:{ QuantitativeValue { value: "30", unit: "C"} }, extent: { TimeInstant { in_datetime: "2015-07-19 17:00:00"} }, relates_to: None } }] } }, { FeatureOfInterest { label: "Wind Direction", Episodes: [{ Episode { semantic_description:{ QualitativeDescription { description: "S"} }, extent: { TimeInstant { in_datetime: "2015-07-19 17:00:00"} }, relates_to: None } }] } }, { FeatureOfInterest { label: "Wind Speed", Episodes: [{ Episode { semantic_description:{ QuantitativeValue { value: "6", unit: "km/h"} }, extent: { TimeInstant { in_datetime: "2015-07-19 17:00:00"} }, relates_to: None } }] } }] }


In [11]:
weather.triplify()

[(rdflib.term.URIRef('http://example.com/resource/nnhVkGU6gVwku9FEFfia2H'),
  rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'),
  rdflib.term.Literal('Weather', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string'))),
 (rdflib.term.URIRef('http://example.com/resource/nnhVkGU6gVwku9FEFfia2H'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('http://purl.org/net/step#ContextualElement')),
 [(rdflib.term.URIRef('http://example.com/resource/rszcWqvFJw7q6UN8fGeH6Z'),
   rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'),
   rdflib.term.Literal('Feels Like', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string'))),
  (rdflib.term.URIRef('http://example.com/resource/rszcWqvFJw7q6UN8fGeH6Z'),
   rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
   rdflib.term.URIRef('http://purl.org/net/step#FeatureOfInterest')),
  (rdflib.term.URIRef('http://example.com/resource/rs

In [12]:
weather.features_of_interest[0]

{ FeatureOfInterest { label: "Feels Like", Episodes: [{ Episode { semantic_description:{ QuantitativeValue { value: "31", unit: "C"} }, extent: { TimeInstant { in_datetime: "2015-07-19 17:00:00"} }, relates_to: None } }] } }

In [13]:
weather.features_of_interest[0].triplify()

[(rdflib.term.URIRef('http://example.com/resource/rszcWqvFJw7q6UN8fGeH6Z'),
  rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'),
  rdflib.term.Literal('Feels Like', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string'))),
 (rdflib.term.URIRef('http://example.com/resource/rszcWqvFJw7q6UN8fGeH6Z'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('http://purl.org/net/step#FeatureOfInterest')),
 (rdflib.term.URIRef('http://example.com/resource/rszcWqvFJw7q6UN8fGeH6Z'),
  rdflib.term.URIRef('http://purl.org/net/step#hasEpisode'),
  rdflib.term.URIRef('http://example.com/resource/hZ9bWFjnAfXPT5vRmw5sN7')),
 (rdflib.term.URIRef('http://example.com/resource/hZ9bWFjnAfXPT5vRmw5sN7'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('http://purl.org/net/step#Episode')),
 (rdflib.term.URIRef('http://example.com/resource/hZ9bWFjnAfXPT5vRmw5sN7'),
  rdflib.term.URIRef('http://p

In [14]:
g = rdflib.Graph()
for triple in weather.triplify():
    g.add(triple)
    
g.serialize("weather.rdf")

ValueError: too many values to unpack (expected 3)